# Background removal tecniques

In [1]:
import numpy as np

In [2]:
import pivimage
import pivtestdata as ptd

from scipy import ndimage

import matplotlib.pyplot as plt

from tqdm import tqdm

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
testdata = ptd.pivtec.vortex_pair
_ = testdata.A[0].plot()

In [ ]:
def normalize(img):

    assert img.ndim == 2
    
    imgN = np.roll(img, (-1, 0))
    imgS = np.roll(img, (1, 0))
    imgE = np.roll(img, (0, 1))
    imgW = np.roll(img, (0, -1))
    
    imgNN = np.roll(imgN, (-1, 0))
    imgSS = np.roll(imgS, (1, 0))
    imgEE = np.roll(imgE, (0, 1))
    imgWW = np.roll(imgW, (0, -1))
    
    imgNE = np.roll(img, (-1, 1))
    imgNW = np.roll(img, (-1, -1))
    imgSE = np.roll(img, (1, 1))
    imgSW = np.roll(img, (1, -1))

    imgMean= np.sum([imgN, imgS, imgW, imgE, imgNE, imgNW, imgSE, imgSW, imgNN, imgSS, imgEE, imgWW], axis=0)/12
    
    imgMean[imgMean==0] = 1e-5
    img_norm  =img/imgMean

    img_norm[:, 1] = 0
    img_norm[1, :] = 0
    img_norm[:, 2] = 0
    img_norm[2, :] = 0

    img_norm[:,-1] = 0
    img_norm[-1, :] = 0
    img_norm[:, -2] = 0
    img_norm[-2, :] = 0

    median_value = np.median(img_norm)
    img_median_norm = img_norm/median_value

    img_median_norm[img_median_norm==0]= 1e-5

    return img_median_norm

def anisotropic_diffusion(img, n_iter=10, lam: float=0.2, K=50):

    imgs = np.empty((n_iter,*img.shape))
    imgs[0, ...] = img
    
    dN = np.empty_like(img)
    dS = np.empty_like(img)
    dE = np.empty_like(img)
    dW = np.empty_like(img)

    ny, nx = img.shape

    for i in tqdm(range(1, n_iter)):
        img = imgs[i-1, ...]
    
        img_normalized = normalize(img)

        # img_med = ndimage.median_filter(img, size=3)
        # img_normalized = img_med / np.median(img_med)
        # img_normalized[img_normalized==0] = 1e-5
        
        
        dy = img_normalized[:-1, :] - img_normalized[1:, :]
        dN[1:,:] = dy
        dN[0,:] = 0 #img[0,:]
    
        dS[:-1,:] = -dy
        dS[-1,:] = img_normalized[-1,:]
    
        dx = img_normalized[:, :-1] - img_normalized[:, 1:]
        dE[:, 1:] = dx
        dE[:,0] = 0 #img[:, 0]
    
        dW[:, :-1] = -dx
        dW[:, -1] = img_normalized[:, -1]
    
        # Computing diffusion coefficients
        
        cN = 1/(1+ (dN/(K*img_normalized))**2)
        cS = 1/(1+ (dS/(K*img_normalized))**2)
        cE = 1/(1+ (dE/(K*img_normalized))**2)
        cW = 1/(1+ (dW/(K*img_normalized))**2)
    
        imgNew = img + lam*(cN*dN + cS*dS + cE*dE + cW*dW)
        
        imgNew[np.isnan(imgNew)] = 0
        imgs[i, ...] = imgNew
    
    return imgs

In [ ]:
img_arr = testdata.A[0].grayscale()[:]

aimg = anisotropic_diffusion(img_arr[:], n_iter=100, K=10, lam=0.20)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 20))
axs[0].imshow(img_arr[:], cmap='gray', vmax=None)
cax = axs[1].imshow(aimg[0], cmap='jet', vmax=None)
plt.colorbar(cax)

In [ ]:
img_arr.max()

In [ ]:
aimg[-1].max()

In [ ]:
_, axs = plt.subplots(1,2)
axs[0].imshow(aimg[0,...], cmap='gray')
axs[1].imshow(aimg[-1,...], cmap='gray')

In [ ]:
#img_arr = pivimage.PIVImage(r'C:/Users/da4323/Documents/programming/live-piv/data/Cam1_0001A.b16')

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(img_arr[:], cmap='gray', vmax=5000)
axs[1].imshow(img_arr[:]-normalize(img_arr[:]), cmap='gray', vmax=5000)

# axs[1].imshow(anisotropic_diffusion(img_arr, n_iter=2, K=3), cmap='gray')

In [ ]:
aimg = anisotropic_diffusion(img_arr[:], n_iter=100, K=10, lam=0.20)

In [ ]:
aimg.max()

In [ ]:
_, axs = plt.subplots(1,2)
axs[0].imshow(aimg[0,...], cmap='gray')
axs[1].imshow(aimg[-1,...], cmap='gray')

In [ ]:
bg = img_arr[:]-aimg[-1,...]

In [ ]:
plt.imshow(img_arr[200:800,200:800], cmap='gray')

In [ ]:
img_arr[:].max()

In [ ]:
plt.imshow(aimg-img_arr[:], cmap='gray')

In [ ]:
aimg[:].max()

In [ ]:
img_arr[:].max()

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(img_arr[:], cmap='gray')

axs[1].imshow(img_arr[:]-aimg, cmap='gray')

In [ ]:
awdawd

In [ ]:
img_arr = testdata.A[0].grayscale()[:]
plt.imshow(img_arr, cmap='gray')

In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(img_arr, cmap='gray')

axs[1].imshow(anisotropic_diffusion(img_arr, n_iter=2, K=3), cmap='gray')

In [ ]:
imgnew = img_arr - anisotropic_diffusion(img_arr, n_iter=100, K=10)

In [ ]:
plt.imshow(imgnew - imgnew.min(), cmap='gray', vmin=0, vmax=255)

In [ ]:
(imgnew - imgnew.min()).max()

In [ ]:
awdawd

In [ ]:
np.sum([imgN._img, imgN._img], axis=0).shape

In [ ]:
imgMean=(imgN+imgS+imgW+imgE+imgNE+imgNW+imgSE+imgSW+imgNN+imgSS+imgEE+imgWW)/12
imgMean(imgMean==0)=10E-05;   
In=img/imgMean;

In [ ]:
import matplotlib.pyplot as plt

img_arr_N = np.roll(img_arr, (-1, 0))

In [ ]:
plt.imshow(img_arr_N)